In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import datetime
import seaborn as sn
from shapely import wkb
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gp
import matplotlib.pyplot as plt
import hashlib
from scipy.spatial.distance import cdist
import os
%matplotlib inline
from collections import deque
import networkx as nx
from time import sleep
from copy import deepcopy
from sympy import Matrix, init_printing
import pickle

In [2]:
matches = pd.read_csv('./test/data-test/mr.csv', sep=';', index_col='id', engine='python')
matches = matches[matches.cpath.isnull()]
# matches['tt'] = matches['ep'].apply(lambda x: np.array([float(e) for e in x.split(',')])) * \
# matches['tp'].apply(lambda x: np.array([float(e) for e in x.split(',')]))
# matches['error'] = matches['error'].apply(lambda x: [float(x) for x in x.split(',')])
# matches['match'] = matches['error'].apply(lambda x: True if all(y<0.00001 for y in x) else False)
# matches = matches[matches['match']]

In [3]:
def geo_center(S, vertices):
    S = list(S)
    mean_point = np.mean(vertices[S], axis=0)
    dists = np.linalg.norm(vertices[S]-mean_point, axis=1)
    center = S[np.argmin(dists)]
    return center

def adjacent_vertex(S, adjacencies):
    out_vertices = set()
    for p in S:
        adjacents = set(np.where(adjacencies[p]==1)[0])
        complements = adjacents-S
        out_vertices = out_vertices.union(complements)
    return out_vertices

def compute_dist(point, seed, dists, predecessors):
    cobweb_dist = 0
    predecessor = predecessors[point]
    while point != seed:
        cobweb_dist += dists[point, predecessor]
        point = predecessor
        predecessor = predecessors[point]
    return cobweb_dist

In [4]:
trajectories = gp.read_file('./test/data-test/trajectories/trajs.shp')
trajectories.set_index('id', drop=True, inplace=True)
trajectories = trajectories[trajectories.index.isin(matches.index)]

In [5]:
a = []
for index, row in trajectories.iterrows():
    for p in list(row.geometry.coords):
        if 35.7388<p[1]<35.740 and 51.3466<p[0]<51.3495:
            a.append(index)

In [6]:
trajectories[trajectories.index.isin(a)]

,geometry
id,
274206777,"LINESTRING (51.34849 35.73968, 51.34871 35.739..."
965437207,"LINESTRING (51.34732 35.73874, 51.34780 35.739..."
794490133,"LINESTRING (51.34668 35.73865, 51.34663 35.739..."
794490133,"LINESTRING (51.34787 35.73436, 51.34783 35.734..."
274206777,"LINESTRING (51.35174 35.73450, 51.35170 35.73451)"
965437207,"LINESTRING (51.34790 35.73427, 51.34789 35.734..."
579481902,"LINESTRING (51.34756 35.74063, 51.34809 35.740..."
577261129,"LINESTRING (51.34678 35.73825, 51.34674 35.738..."
579481902,"LINESTRING (51.34338 35.74049, 51.34495 35.740..."


In [16]:
np.random.seed(123456)
newarray = np.concatenate(np.array(trajectories['geometry'].apply(lambda x: np.array(x.coords))))
Rv = 0.00015
V_set = []
while newarray.size > 0:
    rand_point = newarray[np.random.choice(newarray.shape[0], size=1, replace=False)][0]
    V_set.append(rand_point)
    dists = np.linalg.norm(newarray-rand_point, axis=1)
    newarray = newarray[dists>Rv]
V_set = np.vstack(V_set)

Rc = 0.0009
dists = cdist(V_set, V_set)
adj_matrix = np.ma.getmask(np.ma.masked_where(dists<Rc, dists)).astype(np.int)
np.fill_diagonal(adj_matrix, 0)



r = 0.00085
delta = 5
indices = np.arange(0, V_set.shape[0], 1)
all_marked = set()
all_nodes = set(range(V_set.shape[0]))
random_choices = []
predecessors = {}

while all_nodes != all_marked:
    s = np.random.choice(indices[~np.isin(indices, random_choices)])
    random_choices.append(s)
    Q_g = deque()
    Q_g.appendleft(s)
    Q_l = deque()
    # eddited on the original algorithm
    Q_l.appendleft(s)
    distances = deque()
    distances.appendleft(0)
    count = 0
    S = set()
    marks = np.zeros((V_set.shape[0]), np.int)


#     graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices, 'seed':V_set[s]}
#     with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
#         pickle.dump(graph_dict, graph_file)
#     sleep(2)
    
    marks[s] = 1
    S.add(s)
    print(f'set S is: {S}')

    while len(Q_g) != 0:
        print(f'Q_g currently is: {Q_g}')
        print(f'Q_l currently is: {Q_l}')
        seed = Q_g.pop()
        predecessors[seed] = seed
        print(f'node {seed} choosed as seed')
        print(f'adjacent nodes to seed {seed} are {np.where(adj_matrix[seed]==1)[0]}')
        for p in np.where(adj_matrix[seed]==1)[0]:
            if marks[p] != 1:
                print(f'node {p} accepted as adjacent to seed {seed} and it is unmarked')
                marks[p] = 1
                Q_l.appendleft(p)
                print(f'node {p} marked and appended to Q_l')
                predecessors[p] = seed
                distances.appendleft(compute_dist(p, seed, dists, predecessors))
            else:
                print(f'node {p} accepted as adjacent to seed {seed} but it is marked')
        print(f'adjcent nodes to seed {seed} all appended to Q_l. Now Q_l is : {Q_l}')
        while len(Q_l) != 0:
            p = Q_l.pop()
            print(f'node {p} poped from Q_l')
            temp_dist = distances.pop()
            if temp_dist < r:
                print(f'distance from node {p} to seed {seed} is: {temp_dist} below the threshold {r}') 
                count += 1
                S.add(p)
                print(f'node {p} added to set S. Now S is: {S}')
                print(f'adjacent nodes to {p} are {np.where(adj_matrix[p]==1)[0]}')
                for p_prime in np.where(adj_matrix[p]==1)[0]:
                    if marks[p_prime] != 1:
                        print(f'node {p_prime} accepted as adjacent to {p} and it is unmarked')
                        marks[p_prime] = 1
                        Q_l.appendleft(p_prime)
                        print(f'node {p_prime} marked and appended to Q_l')
                        predecessors[p_prime] = p
                        distances.appendleft(compute_dist(p_prime, seed, dists, predecessors))
                    else:
                        print(f'node {p_prime} accepted as adjacent to {p} but it is marked')
                print(f'adjcent nodes to {p} all appended to Q_l. Now Q_l is : {Q_l}')
                
#                 graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices, 'seed':V_set[seed]}
#                 with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
#                     pickle.dump(graph_dict, graph_file)
#                 sleep(0.5)

            else:
                print(f'distance from node {p} to seed {seed} is: {temp_dist} above the threshold {r}') 
                if count > delta:
                    print(f'number of nodes in S are: {count} above the threshold {delta}')
                    Vc = geo_center(S, V_set)
                    print(f'node {Vc} choosed as the geocenter node')
                    V_prime = adjacent_vertex(S, adj_matrix)
                    print(f'nodes {V_prime} are out of S')
                    ommited = list(S-{Vc})
                    print(f'nodes {ommited} ommited from V except geocenter {Vc}')
                    indices = indices[~np.isin(indices, ommited)]
                    adj_matrix[:, ommited] = 0
                    adj_matrix[ommited] = 0
                    for v in V_prime:
                        Q_g.appendleft(v)
                        print(f'node {v} from V_prime appended to Q_g')
                        adj_matrix[v, Vc] = 1
                        adj_matrix[Vc, v] = 1
                    count = 0
                    S = set()
                    print('set S got empty')
                else:
                    print(f'number of nodes in S are: {count} below the threshold {delta}')

#                 graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices, 'seed':V_set[seed]}
#                 with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
#                     pickle.dump(graph_dict, graph_file)
#                 sleep(0.5)

                # eddited on the original algorithm
                all_marked = all_marked.union(set(np.where(marks==1)[0]))
                while len(Q_l) != 0:
                    p = Q_l.pop()
                    print(f'node {p} poped from Q_l and unmarked')
                    marks[p] = 0
                distances.clear()
                
            all_marked = all_marked.union(set(np.where(marks==1)[0]))
            print('*'*50)
        
        # eddited on the original algorithm
        count = 0
        S = set()
        predecessors = {}
        
    
    all_marked = all_marked.union(set(np.where(marks==1)[0]))
    print(f'{"="*60} all marked are: {all_marked}')
    print(f'{"="*60} all nodes: {all_nodes}')
    print(f'{"="*60} random choices for s: {random_choices}')
    
    
graph_dict = {'marks':marks, 'S':S, 'graph_adj':adj_matrix, 'indices':indices, 'seed':V_set[seed]}
with open('/home/peyman/Documents/projects/balad/codes/test/graph.pkl', 'wb') as graph_file:
    pickle.dump(graph_dict, graph_file)

set S is: {98}
Q_g currently is: deque([98])
Q_l currently is: deque([98])
node 98 choosed as seed
adjacent nodes to seed 98 are [  3   4   7  17  19  24  31  34  51  70  72  74  88 104 130 131 135 154
 158 174]
node 3 accepted as adjacent to seed 98 and it is unmarked
node 3 marked and appended to Q_l
node 4 accepted as adjacent to seed 98 and it is unmarked
node 4 marked and appended to Q_l
node 7 accepted as adjacent to seed 98 and it is unmarked
node 7 marked and appended to Q_l
node 17 accepted as adjacent to seed 98 and it is unmarked
node 17 marked and appended to Q_l
node 19 accepted as adjacent to seed 98 and it is unmarked
node 19 marked and appended to Q_l
node 24 accepted as adjacent to seed 98 and it is unmarked
node 24 marked and appended to Q_l
node 31 accepted as adjacent to seed 98 and it is unmarked
node 31 marked and appended to Q_l
node 34 accepted as adjacent to seed 98 and it is unmarked
node 34 marked and appended to Q_l
node 51 accepted as adjacent to seed 98 an

adjacent nodes to seed 36 are [ 18  43  65  84 108 169 171 190]
node 18 accepted as adjacent to seed 36 but it is marked
node 43 accepted as adjacent to seed 36 but it is marked
node 65 accepted as adjacent to seed 36 but it is marked
node 84 accepted as adjacent to seed 36 but it is marked
node 108 accepted as adjacent to seed 36 but it is marked
node 169 accepted as adjacent to seed 36 but it is marked
node 171 accepted as adjacent to seed 36 but it is marked
node 190 accepted as adjacent to seed 36 but it is marked
adjcent nodes to seed 36 all appended to Q_l. Now Q_l is : deque([])
Q_g currently is: deque([190, 186, 57, 113, 173, 108, 169, 41, 36, 58, 152, 151, 82, 137, 9, 129, 191, 190, 188, 155, 144, 127, 108, 168, 166, 193, 118, 171, 108, 43, 42, 169, 164, 101])
Q_l currently is: deque([])
node 101 choosed as seed
adjacent nodes to seed 101 are [ 18  22  42  65 108 190]
node 18 accepted as adjacent to seed 101 but it is marked
node 22 accepted as adjacent to seed 101 but it is m

node 23 poped from Q_l and unmarked
node 47 poped from Q_l and unmarked
node 48 poped from Q_l and unmarked
node 92 poped from Q_l and unmarked
node 94 poped from Q_l and unmarked
node 142 poped from Q_l and unmarked
node 145 poped from Q_l and unmarked
node 150 poped from Q_l and unmarked
**************************************************
Q_g currently is: deque([189, 92, 150, 145, 81, 47, 11, 39, 2, 1, 189, 54, 48, 47, 45, 172, 39, 96, 94, 92, 153, 23, 150, 81, 145, 142, 11, 1, 64, 54, 45, 172, 39, 28, 26, 153, 21, 20, 16, 13, 10, 8, 6, 5])
Q_l currently is: deque([])
node 5 choosed as seed
adjacent nodes to seed 5 are []
adjcent nodes to seed 5 all appended to Q_l. Now Q_l is : deque([])
Q_g currently is: deque([189, 92, 150, 145, 81, 47, 11, 39, 2, 1, 189, 54, 48, 47, 45, 172, 39, 96, 94, 92, 153, 23, 150, 81, 145, 142, 11, 1, 64, 54, 45, 172, 39, 28, 26, 153, 21, 20, 16, 13, 10, 8, 6])
Q_l currently is: deque([])
node 6 choosed as seed
adjacent nodes to seed 6 are []
adjcent nodes

node 100 accepted as adjacent to seed 63 but it is marked
node 105 accepted as adjacent to seed 63 but it is marked
adjcent nodes to seed 63 all appended to Q_l. Now Q_l is : deque([])
============================================================ all marked are: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 173, 174, 175, 176, 17

============================================================ random choices for s: [98, 128, 91, 188, 180, 190, 185, 44, 22, 130, 159, 148, 133, 24, 115, 170, 92, 38, 1, 97, 30, 42, 113, 192, 129, 64, 156, 58, 173, 2]
set S is: {63}
Q_g currently is: deque([63])
Q_l currently is: deque([63])
node 63 choosed as seed
adjacent nodes to seed 63 are [ 41 111 115 129 152 175 192]
node 41 accepted as adjacent to seed 63 and it is unmarked
node 41 marked and appended to Q_l
node 111 accepted as adjacent to seed 63 and it is unmarked
node 111 marked and appended to Q_l
node 115 accepted as adjacent to seed 63 and it is unmarked
node 115 marked and appended to Q_l
node 129 accepted as adjacent to seed 63 and it is unmarked
node 129 marked and appended to Q_l
node 152 accepted as adjacent to seed 63 and it is unmarked
node 152 marked and appended to Q_l
node 175 accepted as adjacent to seed 63 and it is unmarked
node 175 marked and appended to Q_l
node 192 accepted as adjacent to seed 63 and it i